In [1]:
import rasterio
import rasterio.features
import rasterio.warp
import geopyspark as gps
import numpy as np
import matplotlib.pyplot as plt

from pyspark import SparkContext
from geonotebook.wrappers import TMSRasterData
from osgeo import osr

%matplotlib inline

In [2]:
conf = gps.geopyspark_conf("yarn-client", "CDL Ingest")
conf.set("spark.dynamicAllocation.enabled", False)
conf.set("spark.hadoop.yarn.timeline-service.enabled", False)
#conf.set("spark.kryo.unsafe", True)
#conf.set("spark.rdd.compress", True)
conf.set("spark.yarn.executor.memoryOverhead", "3G")
conf.set("spark.yarn.driver.memoryOverhead", "3G")

In [3]:
sc = SparkContext(conf=conf)

In [4]:
uri = 's3://datahub-rawdata-us-east-1/cdl/CDLS_2016_30m.tif'
# uri = '/tmp/CDLS_2016_30m.tif'

In [5]:
raster_layer = gps.geotiff.get(gps.LayerType.SPATIAL, uri, num_partitions=(32*4))

In [6]:
raster_layer.count()

Py4JJavaError: An error occurred while calling o37.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 2.0 failed 4 times, most recent failure: Lost task 1.3 in stage 2.0 (TID 12, ip-172-31-52-81.ec2.internal, executor 2): java.lang.InstantiationError: java.lang.reflect.InvocationHandler
	at sun.reflect.GeneratedSerializationConstructorAccessor373.newInstance(Unknown Source)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at org.objenesis.instantiator.sun.SunReflectionFactoryInstantiator.newInstance(SunReflectionFactoryInstantiator.java:56)
	at com.esotericsoftware.kryo.Kryo.newInstance(Kryo.java:1090)
	at com.esotericsoftware.kryo.serializers.FieldSerializer.create(FieldSerializer.java:570)
	at com.esotericsoftware.kryo.serializers.FieldSerializer.read(FieldSerializer.java:546)
	at com.esotericsoftware.kryo.Kryo.readObject(Kryo.java:708)
	at com.esotericsoftware.kryo.serializers.ObjectField.read(ObjectField.java:125)
	at com